# Tracing Executions

In this chapter, we show how to observe program state during an execution – a prerequisite for logging and interactive debugging. Thanks to the power of Python, we can do this in a few lines of code.

**Prerequisites**

* You should have read the [Introduction to Debugging](Intro_Debugging.ipynb).
* Knowing a bit of _Python_ is helpful for understanding the code examples in the book.

In [698]:
import bookutils

In [699]:
from bookutils import quiz

## Tracing Python Programs

How do debugging tools access the state of a program during execution? For _interpreted_ languages such as Python, this is a fairly simple task. If a language is interpreted, it is typically fairly easy to control execution and to inspect state – since this is what the interpreter is doing already anyway. Debuggers are then implemented in top of _hooks_ that allow to interrupt execution and access program state.

Python makes such a hook available in the function `sys.settrace()`. You invoke it with a *tracing function* that will be called at every line executed, as in

```python
sys.settrace(traceit)
```

Such a tracing function is convenient, as it simply traces _everything_. In contrast to an interactive debugger, where you have to select which aspect of the execution you're interested in, you can just print out a long trace into an *execution log*, to examine it later.

This tracing function takes the format

In [700]:
def traceit(frame, event, arg):
    ...

Here, `event` is a string telling what has happened in the program – for instance,

* `'line'` – a new line is executed
* `'call'` – a function just has been called
* `'return'` – a fucntion returns

The `frame` argument holds the current execution frame – that is, the function and its local variables:

* `frame.f_lineno` – the current line
* `frame.f_locals` – the current variables (as a Python dictionary)
* `frame.f_code` – the current code (as a Code object), with attributes such as
    * `frame.f_code.co_name` – the name of the current function

We can thus get a *trace* of the program by simply printing out these values:

In [701]:
def traceit(frame, event, arg):
    print(event, frame.f_lineno, frame.f_code.co_name, frame.f_locals)

The return value of the trace function is the function to be executed at the next event – typically, this is the function itself:

In [702]:
def traceit(frame, event, arg):
    print(event, frame.f_lineno, frame.f_code.co_name, frame.f_locals)
    return traceit

Let us try this out on the `remove_html_markup()` function introduced in the [Introduction to Debugging](Intro_Debugging.ipynb):

In [703]:
from Intro_Debugging import remove_html_markup

In [704]:
import inspect
from bookutils import print_content

In [705]:
print_content(content=inspect.getsource(remove_html_markup), filename='.py')

def remove_html_markup(s):
    tag   = False
    quote = False
    out   = ""

    for c in s:
        assert tag or not quote
        
        if c == '<' and not quote:
            tag = True
        elif c == '>' and not quote:
            tag = False
        elif (c == '"' or c == "'") and tag:
            quote = not quote
        elif not tag:
            out = out + c
    
    return out


We define a variant `remove_html_markup_traced()` which turns on tracing, invokes `remove_html_markup()`, and turns tracing off again.

In [706]:
import sys

In [707]:
def remove_html_markup_traced(s):
    sys.settrace(traceit)
    ret = remove_html_markup(s)
    sys.settrace(None)
    return ret

In [708]:
remove_html_markup_traced('xyz')

call 222 remove_html_markup {'s': 'xyz'}
line 223 remove_html_markup {'s': 'xyz'}
line 224 remove_html_markup {'s': 'xyz', 'tag': False}
line 225 remove_html_markup {'s': 'xyz', 'tag': False, 'quote': False}
line 227 remove_html_markup {'s': 'xyz', 'tag': False, 'quote': False, 'out': ''}
line 228 remove_html_markup {'s': 'xyz', 'tag': False, 'quote': False, 'out': '', 'c': 'x'}
line 230 remove_html_markup {'s': 'xyz', 'tag': False, 'quote': False, 'out': '', 'c': 'x'}
line 232 remove_html_markup {'s': 'xyz', 'tag': False, 'quote': False, 'out': '', 'c': 'x'}
line 234 remove_html_markup {'s': 'xyz', 'tag': False, 'quote': False, 'out': '', 'c': 'x'}
line 236 remove_html_markup {'s': 'xyz', 'tag': False, 'quote': False, 'out': '', 'c': 'x'}
line 237 remove_html_markup {'s': 'xyz', 'tag': False, 'quote': False, 'out': '', 'c': 'x'}
line 227 remove_html_markup {'s': 'xyz', 'tag': False, 'quote': False, 'out': 'x', 'c': 'x'}
line 228 remove_html_markup {'s': 'xyz', 'tag': False, 'quote': F

'xyz'

In this very raw format, we can see how the execution progresses through the function. The variable `c` takes one character of the input string at a time; the `out` variable accumulates them. The argument `s` and the `tag` and `quote` flags stay unchanged throughout the execution.

In [709]:
import math

In [710]:
quiz("What happens if the tracing function returns None while tracing function f()? Lookup sys.setttrace() in the Python documentation or try it out yourself.", 
     ['Tracing stops for all functions; the tracing function is no longer called', 
      'Tracing stops for f(): the tracing function is called when f() returns',
      'Tracing stops for f() the rest of the execution: the tracing function is no longer called for calls to f()',
      'Nothing changes'], math.log(7.38905609893065))

## A Tracer Class

Let us refine our tracing function a bit. First, it would be nice if one could actually _customize_ tracing just as needed. To this end, we introduce a `Tracer` class that does all the formatting with us, and which can be _subclassed_ to allow for different output formats.

The `traceit()` method is the same as above, and again is added set up via `sys.settrace()`. Its typical usage, however, is as follows:

```python
with Tracer():
    # Code to be traced
    ...

# Code no longer traced
...
```

When the `with` statement is encountered, the `__enter__()` method is called, which starts tracing. When the `with` block ends, the `__exit__()` method is called, and tracing is turned off. We take special care that the internal `__exit__()` method is not part of the trace, and that any other tracing function that was active before is being restored.

In [711]:
class Tracer(object):
    def __init__(self):
        self.original_trace_function = None
        pass

    def traceit(self, frame, event, arg):
        print(event, frame.f_lineno, frame.f_code.co_name, frame.f_locals)

    def _traceit(self, frame, event, arg):
        if frame.f_code.co_name == '__exit__':
            # Do not trace our own __exit__() method
            pass
        else:
            self.traceit(frame, event, arg)
        return self._traceit

    def __enter__(self):  # Begin of "with" block
        self.original_trace_function = sys.gettrace()
        sys.settrace(self._traceit)

    def __exit__(self, tp, value, traceback):   # End of "with" block
        sys.settrace(self.original_trace_function)

Here's how we use the `Tracer` class. You see that everything works as before, except that it is nicer to use:

In [712]:
with Tracer():
    remove_html_markup("abc")

call 222 remove_html_markup {'s': 'abc'}
line 223 remove_html_markup {'s': 'abc'}
line 224 remove_html_markup {'s': 'abc', 'tag': False}
line 225 remove_html_markup {'s': 'abc', 'tag': False, 'quote': False}
line 227 remove_html_markup {'s': 'abc', 'tag': False, 'quote': False, 'out': ''}
line 228 remove_html_markup {'s': 'abc', 'tag': False, 'quote': False, 'out': '', 'c': 'a'}
line 230 remove_html_markup {'s': 'abc', 'tag': False, 'quote': False, 'out': '', 'c': 'a'}
line 232 remove_html_markup {'s': 'abc', 'tag': False, 'quote': False, 'out': '', 'c': 'a'}
line 234 remove_html_markup {'s': 'abc', 'tag': False, 'quote': False, 'out': '', 'c': 'a'}
line 236 remove_html_markup {'s': 'abc', 'tag': False, 'quote': False, 'out': '', 'c': 'a'}
line 237 remove_html_markup {'s': 'abc', 'tag': False, 'quote': False, 'out': '', 'c': 'a'}
line 227 remove_html_markup {'s': 'abc', 'tag': False, 'quote': False, 'out': 'a', 'c': 'a'}
line 228 remove_html_markup {'s': 'abc', 'tag': False, 'quote': F

## Accessing Source Code

We cam now go and _extend_ the class with additional features. It would be nice if it could actually display the source code of the function being tracked, such that we know where we are. In Python, the function `inspect.getsource()` returns the source code of a function or module. Looking up

```python
module = inspect.getmodule(frame.f_code)
```

gives us the current module, and

```python
inspect.getsource(module)
```

gives us its source code. All we then have to do is to retrieve the current line.

To implement our extended `traceit()` method, we use a bit of a hack. The Python language requires us to define an entire class with all methods as a single, continuous unit; however, we would like to introduce one method after another.  To avoid this problem, we use a special hack: Whenever we want to introduce a new method to some class `C`, we use the construct

```python
class C(C):
    def new_method(self, args):
        pass
```

This seems to define `C` as a subclass of itself, which would make no sense – but actually, it introduces a new `C` class as a subclass of the _old_ `C` class, and then shadowing the old `C` definition.  What this gets us is a `C` class with `new_method()` as a method, which is just what we want.  (`C` objects defined earlier will retain the earlier `C` definition, though, and thus must be rebuilt.)

Using this hack, we can now redefine the `traceit()` method. Our new tracer shows the current line as it is executed.

In [713]:
import inspect

In [714]:
class Tracer(Tracer):
    def traceit(self, frame, event, arg):
        if event == 'line':
            module = inspect.getmodule(frame.f_code)
            if module is None:
                source = inspect.getsource(frame.f_code)
            else:
                source = inspect.getsource(module)
            current_line = source.split('\n')[frame.f_lineno - 2]
            print(frame.f_lineno, current_line)

        return traceit

In [715]:
with Tracer():
    remove_html_markup("abc")

223     tag   = False
224     quote = False
225     out   = ""
227     for c in s:
228         assert tag or not quote
230         if c == '<' and not quote:
232         elif c == '>' and not quote:
234         elif (c == '"' or c == "'") and tag:
236         elif not tag:
237             out = out + c
227     for c in s:
228         assert tag or not quote
230         if c == '<' and not quote:
232         elif c == '>' and not quote:
234         elif (c == '"' or c == "'") and tag:
236         elif not tag:
237             out = out + c
227     for c in s:
228         assert tag or not quote
230         if c == '<' and not quote:
232         elif c == '>' and not quote:
234         elif (c == '"' or c == "'") and tag:
236         elif not tag:
237             out = out + c
227     for c in s:
239     return out


## Tracing Calls and Returns

Next, we'd like to report calling and returning from functions. For the `return` event, `arg` holds the value being returned.

In [716]:
class Tracer(Tracer):
    def traceit(self, frame, event, arg):
        if event == 'call':
            print(f"Calling {frame.f_code.co_name}()")

        if event == 'line':
            module = inspect.getmodule(frame.f_code)
            source = inspect.getsource(module)
            current_line = source.split('\n')[frame.f_lineno - 2]
            print(frame.f_lineno, current_line)

        if event == 'return':
            print(f"{frame.f_code.co_name}() returns {repr(arg)}")

        return traceit

In [717]:
with Tracer():
    remove_html_markup("abc")

Calling remove_html_markup()
223     tag   = False
224     quote = False
225     out   = ""
227     for c in s:
228         assert tag or not quote
230         if c == '<' and not quote:
232         elif c == '>' and not quote:
234         elif (c == '"' or c == "'") and tag:
236         elif not tag:
237             out = out + c
227     for c in s:
228         assert tag or not quote
230         if c == '<' and not quote:
232         elif c == '>' and not quote:
234         elif (c == '"' or c == "'") and tag:
236         elif not tag:
237             out = out + c
227     for c in s:
228         assert tag or not quote
230         if c == '<' and not quote:
232         elif c == '>' and not quote:
234         elif (c == '"' or c == "'") and tag:
236         elif not tag:
237             out = out + c
227     for c in s:
239     return out
remove_html_markup() returns 'abc'


## Tracing Variable Changes

Finally, we'd again like to report variables – but only those that have changed. To this end, we save a copy of the last reported variables in the class, reporting only the changed values.

In [718]:
class Tracer(Tracer):
    def __init__(self):
        self.last_vars = {}

    def changed_vars(self, new_vars):
        changed = {}
        for var_name in new_vars:
            if var_name not in self.last_vars or self.last_vars[var_name] != new_vars[var_name]:
                changed[var_name] = new_vars[var_name]
        self.last_vars = new_vars.copy()
        return changed

Here's how this works: If variable `a` is set to 10 (and we didn't have it so far), it is marked as changed:

In [719]:
t = Tracer()

In [720]:
t.changed_vars({'a': 10})

{'a': 10}

If another variable `b` is added, and only `b` is changed, then only `b` is marked as changed:

In [721]:
t.changed_vars({'a': 10, 'b': 25})

{'b': 25}

If both variables keep their values, nothing changes:

In [722]:
t.changed_vars({'a': 10, 'b': 25})

{}

But if new variables come along, they are listed again.

In [723]:
changes = t.changed_vars({'c': 10, 'd': 25})
changes

{'c': 10, 'd': 25}

The following expression creates a comma-separated list of variables and values:

In [724]:
", ".join([var + " = " + repr(changes[var]) for var in changes])

'c = 10, d = 25'

We can now put all of this together in our tracing function, reporting any variable changes as we see them. Note how we exploit the fact that in a call, all variables have a "new" value; and when we return from a function, we explicitly delete the "last" variables.

In [725]:
class Tracer(Tracer):
    def print_debugger_status(self, frame, event, arg):
        changes = self.changed_vars(frame.f_locals)
        changes_s = ", ".join([var + " = " + repr(changes[var]) for var in changes])

        if event == 'call':
            print("Calling " + frame.f_code.co_name + '(' + changes_s + ')')
        elif changes:
            print(' ' * 40, '#', changes_s)

        if event == 'line':
            module = inspect.getmodule(frame.f_code)
            if module is None:
                source = inspect.getsource(frame.f_code)
            else:
                source = inspect.getsource(module)
            current_line = source.split('\n')[frame.f_lineno - 2]
            print(repr(frame.f_lineno) + ' ' + current_line)

        if event == 'return':
            print(frame.f_code.co_name + '()' + " returns " + repr(arg))
            self.last_vars = {}  # Delete 'last' variables

    def traceit(self, frame, event, arg):
        self.print_debugger_status(frame, event, arg)

Here's the resulting trace of `remove_html_markup()` for a more complex input. You can see that the tracing output allows us to see which lines are executed as well as the variables whose value changes.

In [726]:
with Tracer():
    remove_html_markup('<b>x</b>')

Calling remove_html_markup(s = '<b>x</b>')
223     tag   = False
                                         # tag = False
224     quote = False
                                         # quote = False
225     out   = ""
                                         # out = ''
227     for c in s:
                                         # c = '<'
228         assert tag or not quote
230         if c == '<' and not quote:
231             tag = True
                                         # tag = True
227     for c in s:
                                         # c = 'b'
228         assert tag or not quote
230         if c == '<' and not quote:
232         elif c == '>' and not quote:
234         elif (c == '"' or c == "'") and tag:
236         elif not tag:
227     for c in s:
                                         # c = '>'
228         assert tag or not quote
230         if c == '<' and not quote:
232         elif c == '>' and not quote:
233             tag = False
                          

As you see, even a simple function can create a long execution log. Hence, we will now explore how to focus tracing on particular _events_.

## Conditional Tracing

A log such as the above can very quickly become very messy – notably if executions take a long time, or if data structures become very complex. If one of our local variables were a list with 1,000 entries for instance, and were changed with each line, we'd be printing out the entire list with 1,000 entries for each step. 

We could still load the log into, say, a text editor or a database and then search for specific values, but this is still cumbersome – and expensive. A better alternative, however, is to have our tracer only log while specific _conditions_ hold.

To this end, we introduce a class `ConditionalTracer`, which gets a _conditional expression_ to be checked during executions. Only if this condition holds do we list the current status. With

```python
with ConditionalTracer(condition='c == "z"'):
    remove_html_markup(...)
```

we would obtain only the lines executed while `c` gets a value of `'z'`, and with

```python
with ConditionalTracer(condition='quote'):
    remove_html_markup(...)
```

we would obtain only the lines executed while `quote` is True. If we have multiple conditions, we can combine them into one using `and`, `or`, or `not`.

Our `ConditionalTracer` class stores the condition in its `condition` attribute:

In [727]:
class ConditionalTracer(Tracer):
    def __init__(self, condition=None):
        if condition is None:
            condition = "False"
        self.condition = condition
        self.last_report = None
        super().__init__()

Its `traceit()` function _evaluates_ `condition` and reports the current line only if it holds. To this end, it uses the Python `eval()` function which evaluates the condition in the context of the local variables of the program under test. If the condition gets set, we print out three dots to indicate the elapsed time.

In [728]:
class ConditionalTracer(ConditionalTracer):
    def eval_in_context(self, expr, frame):
        try:
            cond = eval(expr, None, frame.f_locals)
        except NameError:  # (yet) undefined variable
            cond = None
        return cond

The `do_report()` function returns True if the status is to be reported:

In [729]:
class ConditionalTracer(ConditionalTracer):
    def do_report(self, frame, event, arg):
        return self.eval_in_context(self.condition, frame)

We put everything together in our `traceit()` function:

In [730]:
class ConditionalTracer(ConditionalTracer):
    def traceit(self, frame, event, arg):
        report = self.do_report(frame, event, arg)
        if report != self.last_report:
            if report:
                print("...")
            self.last_report = report

        if report:
            self.print_debugger_status(frame, event, arg)

Here's an example. We see that `quote` is set only while the three characters `b`, `a`, and `r` are processed (as should be).

In [731]:
with ConditionalTracer(condition='quote'):
    remove_html_markup('<b title="bar">"foo"</b>')

...
                                         # s = '<b title="bar">"foo"</b>', tag = True, quote = True, out = '', c = '"'
227     for c in s:
                                         # c = 'b'
228         assert tag or not quote
230         if c == '<' and not quote:
232         elif c == '>' and not quote:
234         elif (c == '"' or c == "'") and tag:
236         elif not tag:
227     for c in s:
                                         # c = 'a'
228         assert tag or not quote
230         if c == '<' and not quote:
232         elif c == '>' and not quote:
234         elif (c == '"' or c == "'") and tag:
236         elif not tag:
227     for c in s:
                                         # c = 'r'
228         assert tag or not quote
230         if c == '<' and not quote:
232         elif c == '>' and not quote:
234         elif (c == '"' or c == "'") and tag:
236         elif not tag:
227     for c in s:
                                         # c = '"'
228         assert t

In [759]:
quiz("What happens if the condition contains a syntax error?",
     ["The tracer stops, raising an exception",
      "The tracer continues as if the condition were True",
      "The tracer continues as if the condition were False",
     ], 393 % 7)

Here's the answer, illustrated in two examples. For syntax errors, we indeed get an exception:

In [761]:
with ExpectError():
    with ConditionalTracer(condition='2 +'):
        remove_html_markup('<b title="bar">"foo"</b>')

Traceback (most recent call last):
  File "<ipython-input-761-fac211775813>", line 3, in <module>
    remove_html_markup('<b title="bar">"foo"</b>')
  File "Intro_Debugging.ipynb", line 222, in remove_html_markup
    tag   = False
  File "<ipython-input-711-5f99e6cd406c>", line 14, in _traceit
    self.traceit(frame, event, arg)
  File "<ipython-input-730-e6c101f959a6>", line 3, in traceit
    report = self.do_report(frame, event, arg)
  File "<ipython-input-729-2ae15b24ca05>", line 3, in do_report
    return self.eval_in_context(self.condition, frame)
  File "<ipython-input-733-ae5bdd2a80ba>", line 6, in eval_in_context
    return super().eval_in_context(expr, frame)
  File "<ipython-input-728-bdf362de870d>", line 4, in eval_in_context
    cond = eval(expr, None, frame.f_locals)
  File "<string>", line 1
    2 +
      ^
SyntaxError: unexpected EOF while parsing (expected)


If a variable is undefined, though, the condition evaluates to False:

In [763]:
with ExpectError():
    with ConditionalTracer(condition='undefined_variable'):
        remove_html_markup('<b title="bar">"foo"</b>')

We can also have the log focus on _particular code locations_ only. To this end, we add the pseudo-variables `function` and `line` to our evaluation context, which can be used within our condition to refer to the current function name or line. Then, we invoke the original `eval_cond()` as above.

In [764]:
class ConditionalTracer(ConditionalTracer):
    def eval_in_context(self, expr, frame):
        frame.f_locals['function'] = frame.f_code.co_name
        frame.f_locals['line'] = frame.f_lineno
                    
        return super().eval_in_context(expr, frame)

Again, here is an example. We focus on the parts of the function where the `out` variable is being set:

In [734]:
with ConditionalTracer("function == 'remove_html_markup' and line >= 237"):
    remove_html_markup('xyz')

...
                                         # s = 'xyz', function = 'remove_html_markup', line = 237, tag = False, quote = False, out = '', c = 'x'
237             out = out + c
...
                                         # out = 'x', c = 'y'
237             out = out + c
...
                                         # out = 'xy', c = 'z'
237             out = out + c
...
                                         # line = 239, out = 'xyz'
239     return out
remove_html_markup() returns 'xyz'


Using `line` and `function` in conditions is equivalent to conventional _breakpoints_ in interactive debuggers. We will reencounter them in the next chapter.

In [766]:
quiz("If the program under test contains a variable named 'line', which 'line' does the condition refer to?",
    ["'line' as in the debugger", "'line' as in the program"], (326 * 27 == 8888) + 1)

## Watching Events

As an alternative to conditional logging, we may also be interested to exactly trace when a variable not only _has_ a particular value, but also when it _changes_ its value.

To this end, we set up an `EventTracer` class that _watches_ when some event takes place. It takes a list of expressions ("events") and evaluates them for each line; if any event changes its value, we log the status.

With

```python
with EventTracer(events=['tag', 'quote']):
    remove_html_markup(...)
```

for instance, we would get a listing of all lines where `tag` or `quote` change their value; and with

```python
with EventTracer(events=['function']):
    remove_html_markup(...)
```

we would obtain a listing of all lines where the current function changes.

Our `EventTracer` class stores the list of events in its `events` attribute:

In [735]:
class EventTracer(ConditionalTracer):
    def __init__(self, condition=None, events=[]):
        self.events = events
        self.last_event_values = {}
        super().__init__(condition=condition)

Its `traceit()` function _evaluates_ the individual events and checks if they change.

In [736]:
class EventTracer(EventTracer):
    def events_changed(self, events, frame):
        change = False
        for event in events:
            value = self.eval_in_context(event, frame)

            if (event not in self.last_event_values or 
                value != self.last_event_values[event]):
                self.last_event_values[event] = value
                change = True

        return change

We hook this into `do_report()`, the method that determines whether a line should be shown.

In [737]:
class EventTracer(EventTracer):
    def do_report(self, frame, event, arg):
        return (self.eval_in_context(self.condition, frame) or
                self.events_changed(self.events, frame))

This allows us to track, for instance, how `quote` and `tag` change their values over time.

In [738]:
with EventTracer(events=['quote', 'tag']):
    remove_html_markup('<b title="bar">"foo"</b>')

...
Calling remove_html_markup(s = '<b title="bar">"foo"</b>', function = 'remove_html_markup', line = 222)
...
                                         # line = 224, tag = False
224     quote = False
                                         # line = 225, quote = False
225     out   = ""
...
                                         # line = 227, tag = True, out = '', c = '<'
227     for c in s:
...
                                         # quote = True, c = '"'
227     for c in s:
...
                                         # quote = False
227     for c in s:
...
                                         # tag = False, c = '>'
227     for c in s:
...
                                         # tag = True, out = '"foo"', c = '<'
227     for c in s:
...
                                         # tag = False, c = '>'
227     for c in s:


Continuously monitoring variable values at execution time is equivalent to the concept of *watchpoints* in interactive debuggers.

With this, we have all we need for observing what happens during execution: We can explore the entire state, and we can evaluate conditions and events we are interested in. In the next chapter, we will see how to turn these capabilities into an interactive debugger, where we can query all these things interactively.

## Efficient Tracing

While our framework is very flexible (and can still be extended further), it also is _slow_, since we have to evaluate all conditions and events for every single line of the program. Just how slow are things? We can easily measure this.

In [739]:
from Timer import Timer

In [740]:
runs = 1000

Here's the untraced execution time in seconds:

In [741]:
with Timer() as t:
    for i in range(runs):
        remove_html_markup('<b title="bar">"foo"</b>')
untraced_execution_time = t.elapsed_time()
untraced_execution_time

0.0034735508961603045

And here's the _traced_ execution time:

In [742]:
with Timer() as t:
    for i in range(runs):
        with EventTracer():
            remove_html_markup('<b title="bar">"foo"</b>')
traced_execution_time = t.elapsed_time()
traced_execution_time

0.9146055560559034

We see that the _traced_ execution time is several hundred times slower:

In [743]:
traced_execution_time / untraced_execution_time

263.3056441081422

We can still speed up our implementation somewhat, but still will get nowhere near the untraced execution time.

There is a trick, though, that allows us to execute programs at full speed while being traced. Rather than _dynamically_ checking at run time whether a condition is met, we can also _statically_ inject appropriate code into the program under test. This way, the non-traced code is executed at normal speed.

There is a downside, though: This only works if the condition to be checked is limited to specific _locations_ – because it is precisely these locations where we insert our tracing code. With this limitation, though, _static_ tracing can speed up things significantly.

How does static code injection work? The trick involves _rewriting_ the program code to insert special _debugging statements_ at the given locations. This way, we do not need to use the tracing function at all.

The following `insert_tracer()` function demonstrates this. It takes a function as well as a list of _breakpoint_ lines where to insert tracing statements. At each given line, it injects the code

In [744]:
TRACER_CODE = "TRACER.print_debugger_status(inspect.currentframe(), 'line', None); "

into the function definition, which calls into this tracer:

In [745]:
TRACER = Tracer()

`insert_tracer()` then _compiles_ the resulting code into a new "traced" function, which it then returns.

In [746]:
def insert_tracer(function, breakpoints=[]):
    source_lines, starting_line_number = inspect.getsourcelines(function)

    breakpoints.sort(reverse=True)
    for given_line in breakpoints:
        # Set new source line
        relative_line = given_line - starting_line_number - 1

        inject_line = source_lines[relative_line]
        indent = len(inject_line) - len(inject_line.lstrip())
        source_lines[relative_line] = ' ' * indent + TRACER_CODE + inject_line.lstrip()

    # Rename function
    new_function_name = function.__name__ + "_traced"
    source_lines[0] = source_lines[0].replace(function.__name__, new_function_name)
    new_def = "".join(source_lines)
    
    # For debugging
    print_content(new_def, '.py')
    
    # We keep original source and filename to ease debugging
    prefix = '\n' * starting_line_number    # Get line number right
    new_function_code = compile(prefix + new_def, function.__code__.co_filename, 'exec')
    exec(new_function_code)
    new_function = eval(new_function_name)
    return new_function

Here's an example: inserting two breakpoints in Lines 228 and 239 results in the following (rewritten) definition of `remove_html_markup_traced()`:

In [747]:
remove_html_markup_traced = insert_tracer(remove_html_markup, [228, 239])

def remove_html_markup_traced(s):
    tag   = False
    quote = False
    out   = ""

    for c in s:
        TRACER.print_debugger_status(inspect.currentframe(), 'line', None); assert tag or not quote
        
        if c == '<' and not quote:
            tag = True
        elif c == '>' and not quote:
            tag = False
        elif (c == '"' or c == "'") and tag:
            quote = not quote
        elif not tag:
            out = out + c
    
    TRACER.print_debugger_status(inspect.currentframe(), 'line', None); return out


If we execute the statically instrumented `remove_html_markup_traced()`, we obtain the same output as when using a dynamic tracer. Note that the source code listed shows the original code; the injected calls into `TRACER` do not show up.

In [748]:
with Timer() as t:
    remove_html_markup_traced('<b title="bar">"foo"</b>')
static_tracer_execution_time = t.elapsed_time()

                                         # c = '<', out = '', quote = False, tag = False, s = '<b title="bar">"foo"</b>'
228         assert tag or not quote
                                         # c = 'b', tag = True
228         assert tag or not quote
                                         # c = ' '
228         assert tag or not quote
                                         # c = 't'
228         assert tag or not quote
                                         # c = 'i'
228         assert tag or not quote
                                         # c = 't'
228         assert tag or not quote
                                         # c = 'l'
228         assert tag or not quote
                                         # c = 'e'
228         assert tag or not quote
                                         # c = '='
228         assert tag or not quote
                                         # c = '"'
228         assert tag or not quote
                                         # c = '

How fast is the static tracer compared against the dynamic tracer? This is the execution time of the above code:

In [749]:
static_tracer_execution_time

0.010617839987389743

Compare this against the equivalent dynamic tracer:

In [750]:
with Timer() as t:
    with EventTracer(condition='line == 228 or line == 239'):
        remove_html_markup('<b title="bar">"foo"</b>')
dynamic_tracer_execution_time = t.elapsed_time()
dynamic_tracer_execution_time

...
                                         # s = '<b title="bar">"foo"</b>', function = 'remove_html_markup', line = 228, tag = False, quote = False, out = '', c = '<'
228         assert tag or not quote
...
                                         # tag = True, c = 'b'
228         assert tag or not quote
...
                                         # c = ' '
228         assert tag or not quote
...
                                         # c = 't'
228         assert tag or not quote
...
                                         # c = 'i'
228         assert tag or not quote
...
                                         # c = 't'
228         assert tag or not quote
...
                                         # c = 'l'
228         assert tag or not quote
...
                                         # c = 'e'
228         assert tag or not quote
...
                                         # c = '='
228         assert tag or not quote
...
                                         # c = '"'

0.01675792899914086

In [751]:
dynamic_tracer_execution_time / static_tracer_execution_time

1.578280424176987

We see that the static tracker is several times faster – an advantage that will only increase further as more non-traced code is executed. If our code looks like this:

In [772]:
def some_extreme_function(s):
    ... # Long-running function
    remove_html_markup(s)

and we then execute it with

In [774]:
with EventTracer(condition="function=='remove_html_markup' and line == 239"):
    some_extreme_function("foo")

...
                                         # s = 'foo', function = 'remove_html_markup', line = 239, tag = False, quote = False, out = 'foo', c = 'o'
239     return out
remove_html_markup() returns 'foo'


we will spend quite some time.

In [777]:
quiz("In the above example, where is the EventTracer.traceit() function called?",
     ["When `some_extreme_function()` returns",
      "For each line of `some_extreme_function()`",
      "When `remove_html_markup()` returns",
      "For each line of `remove_html_markup()`"],
     [ord(c) - 100 for c in 'efgh'])

On the other hand, the static tracker is limited to conditions that refer to a _specific location in the code._ If we want to check whether some variable changes, for instance, we have to perform a (nontrivial) static analysis of the code to determine possible locations for a change. If a variable is changed indirectly through references or pointers (a common risk in system-level languages like C), there is no alternative to actually watching its value after each instruction.

## Tracing Binary Executables

Debuggers that act on binary code (say, code compiled from C) operate in a similar way as our "static" tracer: They take a location in the binary code and replace its instruction with a _break instruction_ that interrupts execution, returning control to the debugger. The debugger then replaces the break instruction with the original execution before resuming execution.

If the code cannot be altered (for instance, because it is in read-only memory), however, then debuggers resort to the "dynamic" tracing method, executing one instruction at a time and checking the program counter for its current value after each step.

To provide a minimum of efficient support, some processors, such as x86, provide *hardware breakpoints*. Programmers (or more precisely, debugging tools) can define a set of specific values for the program counter to watch, and if the program counter reaches one of these values, execution is interrupted to return to the debugger. Likewise, *hardware watchpoints* will check specific memory locations at run time for changes and given values. Both hardware watchpoints and hardware breakpoints allow a limited tracking of stopping conditions while still maintaining original execution speed – and the best debugging tools will use a mix of static tracing, dynamic tracing, and hardware tracing.

## Synopsis

This chapter provides a `Tracer()` class that allows to log events during program execution.

The advanced subclass `EventTracer` allows to restrict logs to specific conditions. Logs are shown only while the given `condition` holds:

In [752]:
with EventTracer(condition='line == 223 or len(out) >= 6'):
    remove_html_markup('<b>foo</b>bar')

...
                                         # s = '<b>foo</b>bar', function = 'remove_html_markup', line = 223
223     tag   = False
...
                                         # line = 227, tag = False, quote = False, out = 'foobar', c = 'r'
227     for c in s:
                                         # line = 239
239     return out
remove_html_markup() returns 'foobar'


It also allows to restrict logs to specific events. Log entries are shown only if one of the given `events` changes its value:

In [753]:
with EventTracer(events=["c == '/'"]):
    remove_html_markup('<b>foo</b>bar')

...
Calling remove_html_markup(s = '<b>foo</b>bar', function = 'remove_html_markup', line = 222)
...
                                         # line = 228, tag = False, quote = False, out = '', c = '<'
228         assert tag or not quote
...
                                         # tag = True, out = 'foo', c = '/'
228         assert tag or not quote
...
                                         # c = 'b'
228         assert tag or not quote


`Tracer` and `EventTracer` classes allow for subclassing and further customization.

## Lessons Learned

* Interpreted languages can provide _debugging hooks_ that allow to dynamically control program execution and access program state.
* Tracing can be limited to specific conditions and events:
    * A _breakpoint_ is a condition referring to a particular location in the code.
    * A _watchpoint_ is an event referring to a particular state change.
* Compiled languages allow to _instrument_ code at compile time, injecting code that allows to hand over control to a tracing or debugging tool.

## Next Steps

In the next chapter, we will see how to

* [leverage our tracing infrastructure for interactive debugger](Debugger.ipynb)


## Background

\todo{add}

## Exercises


### Exercise 1: Exception Handling

So far, we have only seen execution of lines in individual functions. But if a function raises an exception, we also may want to catch and report this. Right now, an exception is being raised right through our tracer, interrupting the trace. 

In [754]:
def fail():
    return 2 / 0

In [755]:
with Tracer():
    try:
        fail()
    except:
        pass

Calling fail()
2 def fail():
fail() returns None


Extend the `Tracer` class (or the `EventTracer` subclasses) such that exceptions (event type `'exception'`) are properly traced, too, say as

```
fail() raises ZeroDivisionError: division by zero
```

See the Python documentation for `sys.settrace()`.

**Solution.** Simply extend `print_debugger_status()`:

In [756]:
import traceback

In [757]:
class Tracer(Tracer):
    def print_debugger_status(self, frame, event, arg):
        if event == 'exception':
            exception, value, tb = arg
            print(f"{frame.f_code.co_name}() raises {exception.__name__}: {value}")
        else:
            super().print_debugger_status(frame, event, arg)

In [758]:
with Tracer():
    try:
        fail()
    except:
        pass

Calling fail()
2 def fail():
fail() raises ZeroDivisionError: division by zero
fail() returns None
